In [ ]:
# %%
import matplotlib.pyplot as plt
import numpy as np
import ccfj_rc
from ccfj_rc import GetStationPairs
from geopy.distance import great_circle
import folium
import h5py
import time
import os
from concurrent.futures import ThreadPoolExecutor
import yaml
import pandas as pd
import random
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import plotlib
from toollib_standard import mathlib

In [ ]:
flag_project = 1 # 0--regular ; 1--repartition; 2--voronoi

In [ ]:
if flag_project == 0:
    file_project = 'a-project.yml'
elif flag_project == 1:
    file_project = 'a-project_repar.yml'
elif flag_project == 2:
    file_project = 'a-project_voro.yml'
    
with open(file_project, 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']

In [ ]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

In [ ]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [ ]:
dir_ds = dir_project + info_basic['dir_ds']
dir_CC = dir_CC_workspace + info_basic['name_CC']
dir_partition = dir_project + info_basic['dir_partition']
dir_image = dir_project + info_basic['dir_image'] + 'regular_FJ/'
if os.path.exists(dir_image) == False:
    os.makedirs(dir_image)

In [ ]:
#filename = dir_project+dir_inv_dispernet+ 'inv3.txt'
#nums = np.loadtxt(filename,dtype=int)
nums = [1]
key_subworks_all = info_basic['key_subworks']
key_subworks = []
for num in nums:
    for key_subwork in key_subworks_all:
        if str(num) == key_subwork.split('--')[0]:
            key_subworks.append(key_subwork)
            break
key_subworks = info_basic['key_subworks']
key_subworks

In [ ]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist()
lat_all = stainfo['latitude'].tolist() 
lon_all = stainfo['longitude'].tolist()

In [ ]:
stations = {}
lat = {}
lon = {}
for key in key_subworks:
    filepath = dir_partition + str(key) + '.txt'
    stations_this, lat_stations_this, lon_stations_this = np.loadtxt(filepath, dtype='str' , unpack=True)
    stations[key] = stations_this
    lat[key] = lat_stations_this.astype(float)
    lon[key] = lon_stations_this.astype(float)

In [ ]:
dir_ds = dir_project + info_basic['dir_ds']
dir_disp = dir_project + info_basic['dir_inv_dispernet'] + 'data/'
dir_CC = dir_CC_workspace+info_basic['name_CC']
dir_partition = dir_project + info_basic['dir_partition']
stalistname_all = info_basic['stalistname_all']
dir_image = dir_project + info_basic['dir_image'] + 'bootstrap/'

In [ ]:
name_boot = 'dir_bootstrap/'
dir_boot = dir_project + name_boot
if os.path.exists(dir_boot) == False:
    os.makedirs(dir_boot)
    info_basic['name_boot'] = name_boot
    with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
        yaml.dump(data=info_basic, stream=f, allow_unicode=True)

In [ ]:
ncffile = h5py.File(dir_CC + 'gather_all.h5','r')
ncfs = ncffile['ncfs'][:]
f = ncffile['f'][:]
count_all = ncffile['count'][:]
ncffile.close()

In [ ]:
def time_window_filter(t,ncfst0,r,v_min,t0,a):
    ncfst = ncfst0.copy()
    for i in range(len(ncfst)):
        tag = r[i]/v_min
        #print(t0,tag)
        t1 = t[t>-tag-t0][t[t>-tag-t0]< tag+t0]
        start = np.where(t == t1[0])[0][0]
        end = np.where(t == t1[-1])[0][0]
        ncfst[i][start:end+1]= ncfst[i][start:end+1]* np.exp(-a*np.abs((tag-np.abs(t1))))
    return ncfst

In [ ]:
def remove_stack(ncfs_sum_linear,r):
    global dir_project
    global key_subworks
    global info_basic
    global flag_save
    global v_tag
    global t0
    global a

    f0 = info_basic_bi['f']
    dt = 1/np.max(f0)
    t = (np.linspace(-len(f0)-1,len(f0)-1,2*(len(f0)-1))+0.5)*dt/2
    ncfst_linear = mathlib.freq_time(ncfs_sum_linear)
    ncfst1 = time_window_filter(t,ncfst_linear,r,v_tag,t0,a)
    ncfs_sum_remove = mathlib.time_freq(ncfst1)
    return ncfs_sum_remove

In [ ]:
ds_linear_all = {}
ds_remove_all = {}
index_ncfs = {}
rs = {}
for key_subwork in key_subworks:
    print(key_subwork+ '; '+str(key_subworks.index(key_subwork)+1)+'/'+str(len(key_subworks))+' subworks.')
    ds = h5py.File(dir_ds+'ds_'+str(key_subwork)+'.h5', 'r')
    ds_linear_all[key_subwork] = ds['ds_linear'][0]
    ds_remove_all[key_subwork] = ds['ds_remove'][0]
    index_ncfs[key_subwork] = list(ds['index_ncfs'])
    rs[key_subwork] = np.array(ds['r'])
    ds.close()

In [ ]:
flag_plot = 1
flag_save = 0
# remove parameter
v_tag = info_basic['v_tag']
t0 = info_basic['t0']
a = info_basic['a']
# bootstrap parameter
inter_f = 10
start_f = 5
end_f = 20
random_num=20
f0 = info_basic_bi['f']
index_start_f = np.argmin(np.abs(f0-start_f)) # 300
index_end_f = np.argmin(np.abs(f0-end_f)) # 700
c = np.linspace(info_basic['c_min'],info_basic['c_max'],info_basic['c_num'])
nf = len(f)
nc = len(c)

In [ ]:
start00 = time.time()
for key_subwork in key_subworks:
    start0 = time.time()
    print('Start '+ key_subwork+' ...')
    ds_this = ds_remove_all[key_subwork]
    index = index_ncfs[key_subwork]
    ncfs_sum_linear = ncfs[index,:]
    r = rs[key_subwork]
    nr = len(r)
    nr_selected = int(nr/2)
    
    stalist = stations[key_subwork]
    lat_this = lat[key_subwork]
    lon_this = lon[key_subwork]
    nsta = len(stalist)
    StationPairs = GetStationPairs(nsta)
    nPairs = int(len(StationPairs)/2)
    ncfs_sum_remove = remove_stack(ncfs_sum_linear,r)
    ds_remove = plotlib.smooth_ds(ncfs_sum_remove)

    # random pick
    random_indx = np.array(random.sample(range(nr), nr_selected))
    ncfs_new = ncfs_sum_remove[random_indx,:]
    r1 = r[random_indx]
    indx = np.argsort(r1)
    r1 = r1[indx]
    ncfs_new = ncfs_new[indx]

    # read dispersion curves
    file_curve = dir_disp + 'ds_'+key_subwork+'curve.txt'
    curve = np.loadtxt(file_curve)
    curve0 = curve[curve[:,2]==0,:]
    c0 = np.interp(f0, curve0[:,0], curve0[:,1])
    fr = f0[index_start_f:index_end_f:inter_f]
    cr = c0[index_start_f:index_end_f:inter_f]

    # redefine calculate zone
    c = np.zeros(nf*2)
    for i in range(nf):
        c[i*2] = c0[i]*0.93
        c[i*2+1] = c0[i]*1.07
    F = np.zeros([nc,nf])
    C = np.zeros([nc,nf])
    for i in range(nf):
        for j in range(nc):
            F[j,i] = f0[i]
            C[j,i] = c[i*2]+(c[i*2+1]-c[i*2])/(nc-1)*j
    ds_new = ccfj_rc.fj_noise(np.real(ncfs_sum_remove),r,c,f0,nc,func=1)

    # center max
    cr0 = np.zeros_like(fr)
    ds_smooth = plotlib.smooth_ds(ds_new)
    ds_smooth = np.nan_to_num(ds_smooth)
    for i in range(len(fr)):
        j = index_start_f+i*inter_f
        idx = np.argmax(ds_smooth[:,j])
        cr0[i] = C[idx,j]

    # calculate bootstrap
    cr1 = np.zeros([random_num,len(fr)])
    for i in range(random_num):
        #print(i)
        random_indx = np.array(random.sample(range(nr), nr_selected))
        ncfs1 = ncfs_sum_remove[random_indx,:]
        r1 = r[random_indx]
        indx = np.argsort(r1)
        r1 = r1[indx]
        ncfs1 = ncfs1[indx]
        ds = ccfj_rc.fj_noise(np.real(ncfs1),r1,c,f,nc,func=1)
        for j in range(len(fr)):
            k = index_start_f+j*inter_f	
            idx = np.argmax(ds[:,k])
            cr1[i,j] = C[idx,k]
    
    # Save
    if flag_save:
        outname = key_subwork+'_bootstrap.h5'
        if os.path.exists(dir_boot+outname):
            os.remove(dir_boot+outname)
        h5file = h5py.File(dir_boot+outname,'w')
        h5file.create_dataset('fr0',data=fr)
        h5file.create_dataset('cr0',data=cr0)
        h5file.create_dataset('cr1',data=cr1)
        h5file.create_dataset('F',data=F)
        h5file.create_dataset('C',data=C)
        h5file.create_dataset('ds_remove',data=ds_remove)
    
    print('Finish *** '+ key_subwork + ' ***,',time.time()-start0, ' seconds; time since start:', time.time()-start00, ' seconds. Proceeded '+str(key_subworks.index(key_subwork)+1)+'/'+str(len(key_subworks))+' subworks.')

    # plot
    if flag_plot == 1:
        fig = plt.figure(figsize=(10, 7))
        plt.pcolormesh(F,C,ds_smooth,vmin=0.5,cmap='jet')
        for i in range(random_num):
            plt.scatter(fr,cr1[i,:],c='gray',marker='.',s = 3)
        plt.plot(fr,cr0,'k^',markersize=3)

        c_max = []
        c_min = []
        for i in range(len(fr)):
            c_max.append(np.max(cr1[:,i]))
            c_min.append(np.min(cr1[:,i]))
        plt.plot(fr,c_max,'b--')
        plt.plot(fr,c_min,'b--')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Phase Velocity (km/s)')
        plt.title(key_subwork)
        plt.tight_layout()
        if os.path.exists(dir_image)==False:
            os.makedirs(dir_image)
        plt.savefig(dir_image+key_subwork+'_bootstrap')
        plt.close()